# Generate Training Data

We wish to speed up the process of having a human being name entities that they recognize in a paragraph, and having their positions in the paragraph identified and placed in a syntax usable by spaCy's NER training routine.

## Open PDF file, extract page two and display as sentences

In [13]:
import spacy
import PyPDF2
nlp = spacy.load('en_core_web_sm')

#Open PDF file for reading
pdfFile = open("BarCvDescLJ11.pdf", mode="rb")
pdfReader = PyPDF2.PdfFileReader(pdfFile)

# Select a page to work on
pageNumber = 29

# Get text
OnePage = pdfReader.getPage(pageNumber-1) #0-based count
OnePageText = OnePage.extractText()

# Close PDF file
pdfFile.close()

# Remove newlinesxmx82k-&a. It appears multiple newlines together makes
# Spacy think that is the end of a sentence. The PDF reader reads the text in
# an odd fashion
OnePageText = OnePageText.replace('\n','')

# create a spaCy doc object from the page and break it into sentences
doc = nlp(OnePageText)
l=0
for sent in doc.sents:
     print(l, ": ", sent)
     l = l+1


0 :  29   
1 :  ROBUST  Robust is a six-rowed spring malting barley.
2 :  It was released by the Minnesota AES in 1983.
3 :  It was selected from the cross Morex/Manker.
4 :  Its experimental designation was M36.
5 :  It is adapted to barley growing areas in the upper Midwest.
6 :  It is smooth-awned with medium early maturity (matures 2-3 days earlier than Morex).
7 :  Plants are mid-tall with moderately strong straw.
8 :  Spikes are medium-lax, medium-long and semi-erect.
9 :  Kernels are covered, medium-large with short haired rachilla and white aleurone.
10 :  The central vein is moderately prominent, lateral veins are less prominent and disappearing near the center of the kernel.
11 :  There are no barbs on the lateral veins.
12 :  The crease is V-shaped and narrow at the base.
13 :  Lateral kernels are moderately twisted, plump, wide at the center, and full on the crease side.
14 :  It is superior to Morex in grain yield, kernel plumpness and lodging resistance.
15 :  At the time

## Read in Per-line named entity file and match entities to sentence positions.

In [15]:
import re
import csv
import pandas as pd
fname = "Data/barley_p29_ner.txt"

# Covert the nlp senetence generator into a list of sentences
sentences = list(doc.sents)

# Open the file of manually matched pairs (sentence # <tab> word phrase <tab> named entity)
# e.g.:
#  0      AC Metcalfe     CVAR
#  0      two-rowed       TRAT
#  0      barley          CROP
#  1      Agri-Food Candada   ORG
#  1      1997    DATE
file = open(fname)
reader = csv.reader(file, delimiter='\t', quoting=csv.QUOTE_NONE)
data = list()

for row in reader:
    try:
        (sentIndex, phrase, label) = row
        sent = sentences[int(sentIndex)].string.rstrip()
        
        # find all instances of the 'phrase' in the 'sent'.
        iter = re.finditer(r"\b"+phrase+r"\b", sent)
        indices = [m.start(0) for m in iter]
        
        # check to make sure the phrase the user said was there was indeed found
        if len(indices) == 0:
            raise ValueError
                
        # print out all instances
        for i in indices:
#            print(sentIndex, sent, phrase, "("+str(i), i+len(phrase), "'"+label+"')")
            data.append([sentIndex, sent, phrase, "("+str(i)+", "+str(i+len(phrase))+", '"+label+"')"])
            
    except:
        print("Handle manually: ", row)
        
df = pd.DataFrame(data, columns = ["Index", "Sentence", "Phrase", "MatchInfo"])
print(df)


Handle manually:  ['17', 'Crop Science 23:1216 (1983)', 'JRNL']
Handle manually:  ['20', 'Bracken/UT75B65-532 (ID633019/Woodvale)', 'PED']
Handle manually:  ['32', 'Crop Science 33:1412-1413 (1993)', 'JRNL']
Handle manually:  ['52', 'Crop Science 28:574 (1988)', 'JRNL']
    Index                                           Sentence       Phrase  \
0       1  ROBUST  Robust is a six-rowed spring malting b...       Robust   
1       1  ROBUST  Robust is a six-rowed spring malting b...    six-rowed   
2       1  ROBUST  Robust is a six-rowed spring malting b...       spring   
3       1  ROBUST  Robust is a six-rowed spring malting b...      malting   
4       1  ROBUST  Robust is a six-rowed spring malting b...       barley   
..    ...                                                ...          ...   
147    63  Sara is a stripe rust resistant hooded spring ...  stripe rust   
148    63  Sara is a stripe rust resistant hooded spring ...    resistant   
149    63  Sara is a stripe rust res

## Create a function to clean up overlapping intervals

In [16]:
import re
coordRegex = re.compile(r'(\d+), (\d+)')

def sortByStart(coords):
    """For use in sort routines, return object with lowest (X,Y) values"""
    # split out coordinates that come in as (5, 7, 'CVAR')
    mo = coordRegex.search(coords)
    return(int(mo.group(1)))

def overlaps(coord1, coord2):
    """Check if coordinates of the form 5, 7, 'CVAR' and 32, 46, 'TRAT' overlap"""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
    
    if ((coord1High >= coord2Low) and (coord1Low <= coord2Low) or
        (coord2High >= coord1Low) and (coord2Low <= coord1Low)):
        return True
    else:
        return False

def keepFirst(coord1, coord2):
    """Given overlapping coordinates, return the wider encompassing one."""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
 
    if (int(coord1High) - int(coord1Low)) >= (int(coord2High) - int(coord2Low)):
        return True
    else:
        return False

# print("Should be false:", overlaps("(5, 7, 'CVAR')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(26, 46, 'TRAT')"))
# print("Keeper:", keepFirst("34, 46, 'TRAT'", "34, 46, 'TRAT'"))

def cleanIntervals(inputString=""):
    """order intervals like (5, 7, 'CVAR'), (32, 46, 'TRAT'), (26, 46, 'TRAT') and remove overlapping ones."""
    inputString = inputString.lstrip("(").rstrip(")")
    intervalList = inputString.split("), (")
    intervalList.sort(key = sortByStart)
#    print("Sorted Interval List:", intervalList)

    # Pairwise compare every interval in the list to every other interval to check overlap
    keeperList = [True]*len(intervalList) # Logic array to determine if each interval should be kept
    i=0
    for interval1 in intervalList:
        for interval2 in intervalList:
            if interval1 == interval2:
                if intervalList.index(interval1) != i: # when both are the same we reject the higher one
                    keeperList[i] = False
            else:
                if overlaps(interval1, interval2) and keepFirst(interval1, interval2) == False:
                    keeperList[i] = False
        i = i+1
        
#    print("keeperList:", keeperList)
   
    # Build up the return interval list
    returnStr = "("
    for interval, isKeeper in zip(intervalList, keeperList):
        if isKeeper:
            returnStr = returnStr + interval + "), ("
    return (returnStr.rstrip("), (") + ")")
        
# cleanIntervals("(5, 7, 'CVAR'), (32, 46, 'TRAT'), (5, 9, 'CVAR'), (48, 55, 'ORG'), (26, 46, 'TRAT')")
# cleanIntervals("(0, 8, 'CVAR'), (0, 5, 'CVAR'), (21, 26, 'PLAN'), (32, 37, 'CVAR')")
cleanIntervals("(0, 12, 'CVAR'), (39, 49, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (140, 150, 'CVAR'), (181, 187, 'CVAR'), (181, 187, 'CVAR')")


"(0, 12, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (181, 187, 'CVAR')"

## Aggregate all matches for each sentence on a single line and output in spaCy training format

In [17]:
# use Pandas dataframes to aggregate all entity matches together for a single sentence
agg_rules = {'Sentence': 'first', 'Phrase': 'first', 'MatchInfo': lambda x: ', '.join(x)}
res = df.groupby('Index').agg(agg_rules)
#print(res)

# Now format it just like what is needed for the spaCy training module: 
# E.g.:
# ('Eight-Twelve is a six-rowed winter feed barley', {'entities': [(0, 12, 'CVAR'), (18, 27, 'TRAT'), (28, 39, 'TRAT'),(40, 46, 'CROP')]}),
records = res.to_dict('records')
for i in range(0,len(records)):
    print("    ('"+records[i]['Sentence']+"', {'entities': ["+cleanIntervals(records[i]['MatchInfo'])+"]}),")
    #print("    ('"+records[i]['Sentence']+"', {'entities': ["+records[i]['MatchInfo']+"]}),")


    ('ROBUST  Robust is a six-rowed spring malting barley.', {'entities': [(8, 14, 'CVAR'), (20, 29, 'TRAT'), (30, 36, 'TRAT'), (37, 44, 'TRAT'), (45, 51, 'CROP')]}),
    ('The central vein is moderately prominent, lateral veins are less prominent and disappearing near the center of the kernel.', {'entities': [(4, 16, 'PLAN'), (42, 55, 'PLAN'), (115, 121, 'PLAN')]}),
    ('There are no barbs on the lateral veins.', {'entities': [(13, 18, 'PLAN'), (26, 39, 'PLAN')]}),
    ('The crease is V-shaped and narrow at the base.', {'entities': [(4, 10, 'PLAN'), (41, 45, 'PLAN')]}),
    ('Lateral kernels are moderately twisted, plump, wide at the center, and full on the crease side.', {'entities': [(0, 15, 'PLAN'), (83, 89, 'PLAN')]}),
    ('It is superior to Morex in grain yield, kernel plumpness and lodging resistance.', {'entities': [(18, 23, 'CVAR'), (27, 38, 'TRAT'), (40, 56, 'TRAT'), (61, 79, 'TRAT')]}),
    ('At the time of release it was highly resistant to stem rust and spot blotch and s